    This file is a part of the lor_neuro_rat project.
    Copyright (C) 2019 anonimous

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

    Please contact with me by E-mail: shkolnick.kun@gmail.com

# Подготовка удаленных сообщений и причин удаления для разметки.

In [ ]:
import pandas as pd
#Загрузка
df = pd.read_pickle('data/all_data_as_dataframe.pkl')
#Сортировка
df.sort_values(by=['DelReason'], inplace=True, ascending=False)
df.reset_index(inplace=True, drop=True)

print(df.head())

In [ ]:
import re
#Удаленные сообщения
DeletedMsg = df[df['DelReason'] != '']
#Создатель
Creator = DeletedMsg['Creator'].apply(lambda x: x.split('/')[2] if x else x)
#Кто удалил
DeletedBy = DeletedMsg['DelReason'].apply(lambda x: re.sub('Сообщение удалено ', '', x).split()[0] if x else '')
#Удаленные модераторами
DeletedByMod = DeletedMsg[Creator != DeletedBy]

In [ ]:
DeletedByMod.head()


In [ ]:
MsgOfInterest = DeletedByMod.copy()

In [ ]:
print(len(MsgOfInterest))
for r in MsgOfInterest['DelReason']:
    if 'по причине' not in r:
        print(r)

Всего одно сообщение, ну и ладно

In [ ]:
MsgOfInterest = MsgOfInterest[MsgOfInterest['DelReason'].apply(lambda x: 'по причине' in x)].copy()

In [ ]:
Reason = MsgOfInterest['DelReason'].apply(lambda x: x.split('по причине')[1])

In [ ]:
for r in MsgOfInterest['DelReason']:
    try:
        a = r.split('по причине')[1].split()[-1].split('(')[1]
    except Exception as e:
        print(e)
        print(r)

In [ ]:
def get_score(x):
    try:
        score = int(x.split('по причине')[1].split()[-1].split('(')[1].split(')')[0])
    except Exception as e:
        print(e)
        print(x)
        score = 0
    return score

Score = MsgOfInterest['DelReason'].apply(get_score)

In [ ]:
#Описания причин удаления, дальше будем размечать lookup table вручную
import re
"""
reasons = ['7.1 Ответ на некорректное сообщение',
           '3.2 Неверная кодировка',
           '3.3 Некорректное форматирование',
           '3.4 Пустое сообщение',
           '4.2 Вызывающе неверная информация']
"""

reasons = []
for r in Reason:
    r = re.sub(r',', ';', r)
    if r not in reasons:
        reasons.append(r)
reasons.sort()

reason_lut = pd.DataFrame({'Reason':reasons, 'Class':[0]*len(reasons)})
reason_lut.to_csv('data/markup/reasons_lut.csv', index=False, sep=',')

In [ ]:
print(len(MsgOfInterest))
print(len(Reason))
print(len(Score))

In [ ]:
MsgOfInterest['Reason'] = Reason.values
MsgOfInterest['Score'] = Score.values
MsgOfInterest.to_pickle('data/MsgOfInterestDataFrame.pkl')